In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stops = stopwords.words("russian")
stops.extend(['что', 'это', 'так', 'вот', 'быть', 'как', 'в', 'к', 'на'])

In [6]:
data = pd.read_csv('data.csv', sep=';')
vectorizer = TfidfVectorizer(max_features=25000)
tfidf = vectorizer.fit(data['plot'])
X = vectorizer.transform(data['plot']).toarray()
pickle.dump(tfidf, open('tfidf.pickle', 'wb'))
pickle.dump(X, open('tfidf_matrix.pickle', 'wb'))    # https://yadi.sk/d/tbhfd_gJY41ofg

def predict(query):
    query = ' '.join(list(morph.parse(word)[0].normal_form for word in re.findall("[\w]+", query)))
    query = ' '.join([word for word in query.split() if word not in stops])
    vec = vectorizer.transform([query]).toarray().reshape(25000, 1)
    data = pd.read_csv('data.csv', sep=';')
    simil = []
    for vector, film in zip(X, data['film']):
        simil.append((np.dot(vector, vec)/np.linalg.norm(vector)*np.linalg.norm(vec), film))
    simil.sort(reverse=True)
    return simil[:5]

query = 'Ким Ки Тхэк раньше работал водителем, но давно уже потерял работу и теперь влачит жалкое существование вместе со своей женой Чхун Сук и двумя детьми выпускного возраста в полуподвальной квартире. Семья пытается свести концы с концами, складывая из картонок коробки для пиццы.'
#query = 'Сериал про Шерлока Холмса'
predict(query)

[(array([0.36530307]), 'Драйв (фильм, 2011)'),
 (array([0.34616258]), 'Паразиты (фильм)'),
 (array([0.33944276]), 'Гарем Степана Гуслякова'),
 (array([0.33806448]), 'Водитель (фильм)'),
 (array([0.33272957]), 'Вся семья работает')]